In [1]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import pandas as pd
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from skimage import io
from skimage.filters import frangi
from PIL import ImageFilter
from skimage.color import rgb2gray

class CustomImageDataset(Dataset):
    def __init__(self, img_dir, csv_file, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.dataframe = pd.read_csv(csv_file)
        self.classes = self.dataframe.iloc[:, 1].unique()

    def __len__(self):
        return len(self.dataframe)
    # ...

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.dataframe.iloc[idx, 0] + '.jpg')

        image = Image.open(img_name)  # Use PIL.Image.open
        image = image.resize((64, 64))  # Resize the image

        image = np.array(image)  # Convert the image to a numpy array

        # Check the dimensions of the image
        if len(image.shape) < 2 or len(image.shape) > 3:
            print(f"Invalid image dimensions for image {img_name}: {image.shape}")
            return

        # Convert to grayscale if necessary
        if len(image.shape) == 3 and image.shape[2] > 1:
            image = rgb2gray(image)  # RGB to grayscale

        # Apply the Frangi Vessel filter
        image = frangi(image)

        image = (image * 255).astype(np.uint8)  # Convert to uint8
        image = Image.fromarray(image)
        label = self.dataframe.iloc[idx, 1]
        label = int(label)

        if self.transform:
            image = self.transform(image)

        return image, label


# Mount Google Drive
drive.mount('/content/drive')

# Define paths to datasets
train_dir = '/content/drive/MyDrive/Disease_Grading/Training_Set'
test_dir = '/content/drive/MyDrive/Disease_Grading/Testing_Set'
train_csv = '/content/drive/MyDrive/Disease_Grading/Training_Labels.csv'
test_csv = '/content/drive/MyDrive/Disease_Grading/Testing_Labels.csv'

# Define transforms with data augmentation
transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Create datasets and data loaders
trainset = CustomImageDataset(train_dir, train_csv, transform=transform)
testset = CustomImageDataset(test_dir, test_csv, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=True)

# Enable anomaly detection
torch.autograd.set_detect_anomaly(True)


#CALL YOUR MODEL HERE
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters())

# Train the model
n_epochs = 30
for epoch in range(n_epochs):
    total_loss = 0
    correct = 0
    total = 0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model(images)

        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        # Compute training accuracy
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        total_loss += loss.item()

    accuracy = 100 * correct / total
    average_loss = total_loss / len(trainloader)

    print(f"Epoch {epoch+1} / {n_epochs} : Loss {average_loss:.4f}, Accuracy: {accuracy:.2f}%")


ModuleNotFoundError: No module named 'google.colab'